In [3]:
def route_cipher(text, password, m, n, fill_char='а'):

    text = text.replace(' ', '').upper()
    password = password.upper()
    
    block_size = m * n
    if len(text) % block_size != 0:
        padding = block_size - (len(text) % block_size)
        text += fill_char.upper() * padding
    
    password_order = sorted([(char, i) for i, char in enumerate(password)])
    column_order = [idx for char, idx in password_order]
    
    result = []
    
    for block_start in range(0, len(text), block_size):
        block = text[block_start:block_start + block_size]
        
        matrix = []
        for i in range(m):
            row_start = i * n
            row_end = row_start + n
            matrix.append(list(block[row_start:row_end]))
        
        for col_idx in column_order:
            for row in range(m):
                result.append(matrix[row][col_idx])
    
    return ''.join(result)

In [4]:
def route_decipher(ciphertext, password, m, n):
    
    password = password.upper()
    block_size = m * n
    
    password_order = sorted([(char, i) for i, char in enumerate(password)])
    column_order = [idx for char, idx in password_order]
    
    inverse_order = [0] * n
    for new_pos, old_pos in enumerate(column_order):
        inverse_order[old_pos] = new_pos
    
    result = []

    for block_start in range(0, len(ciphertext), block_size):
        block = ciphertext[block_start:block_start + block_size]

        matrix = [[''] * n for _ in range(m)]
        
        idx = 0
        for col_idx in column_order:
            for row in range(m):
                if idx < len(block):
                    matrix[row][col_idx] = block[idx]
                    idx += 1

        for row in range(m):
            for col in range(n):
                result.append(matrix[row][col])
    
    return ''.join(result)

In [5]:
test_text = "нельзя недооценивать противника"
password_route = "пароль"
password_grid = "шифр"
password_vigenere = "математика"


In [6]:
encrypted_route = route_cipher(test_text, password_route, 5, 6)
decrypted_route = route_decipher(encrypted_route, password_route, 5, 6)

In [8]:

print(f"После шифрования: {encrypted_route}")
print(f"После расшифровки: {decrypted_route}")
print()

После шифрования: ЕЕНПНЗОАТАЬОВОКННЕЬВЛДИРИЯЦТИА
После расшифровки: НЕЛЬЗЯНЕДООЦЕНИВАТЬПРОТИВНИКАА



In [2]:
def create_grid(k):

    base_grid = [[(i * k + j + 1) for j in range(k)] for i in range(k)]

    full_grid = [[0] * (2*k) for _ in range(2*k)]

    positions = []

    for i in range(k):
        for j in range(k):
            positions.append((i, j, base_grid[i][j]))

    for i in range(k):
        for j in range(k):
            positions.append((j, k-1-i + k, base_grid[i][j]))

    for i in range(k):
        for j in range(k):
            positions.append((k-1-i + k, k-1-j + k, base_grid[i][j]))

    for i in range(k):
        for j in range(k):
            positions.append((k-1-j + k, i, base_grid[i][j]))
    
    return positions


In [ ]:
def grid_cipher(text, password, k, fill_char='а'):

    text = text.replace(' ', '').upper()
    password = password.upper()

    positions = create_grid(k)

    grid_size = 2 * k
 
    if len(text) < grid_size * grid_size:
        padding = grid_size * grid_size - len(text)
        text += fill_char.upper() * padding

    grid = [[''] * grid_size for _ in range(grid_size)]

    char_idx = 0
    for rotation in range(4):

        current_positions = [pos for pos in positions if 1 <= pos[2] <= k*k]

        current_positions.sort(key=lambda x: x[2])
  
        for i, j, num in current_positions:
            if char_idx < len(text):
                grid[i][j] = text[char_idx]
                char_idx += 1

    password_order = sorted([(char, i) for i, char in enumerate(password[:grid_size])])
    column_order = [idx for char, idx in password_order]
    
    result = []
    for col_idx in column_order:
        for row in range(grid_size):
            result.append(grid[row][col_idx])
    
    return ''.join(result)

In [9]:
def vigenere_cipher(text, key, alphabet=None):

    if alphabet is None:
        alphabet = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
    
    text = text.replace(' ', '').upper()
    key = key.upper()
    
    table = []
    for i in range(len(alphabet)):
        row = alphabet[i:] + alphabet[:i]
        table.append([char.upper() for char in row])
    
    alphabet_upper = alphabet.upper()
    
    result = []
    key_idx = 0
    
    for char in text:
        if char in alphabet_upper:

            row_idx = alphabet_upper.index(char)

            key_char = key[key_idx % len(key)]
            col_idx = alphabet_upper.index(key_char)
            
            cipher_char = table[row_idx][col_idx]
            result.append(cipher_char)
            
            key_idx += 1
        else:
            result.append(char)
    
    return ''.join(result)

In [11]:
def vigenere_decipher(ciphertext, key, alphabet=None):

    if alphabet is None:
        alphabet = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
    
    key = key.upper()
    alphabet_upper = alphabet.upper()

    table = []
    for i in range(len(alphabet)):
        row = alphabet[i:] + alphabet[:i]
        table.append([char.upper() for char in row])
    
    result = []
    key_idx = 0
    
    for char in ciphertext:
        if char in alphabet_upper:

            key_char = key[key_idx % len(key)]
            col_idx = alphabet_upper.index(key_char)

            row_idx = -1
            for i in range(len(alphabet)):
                if table[i][col_idx] == char:
                    row_idx = i
                    break

            plain_char = alphabet_upper[row_idx]
            result.append(plain_char)
            
            key_idx += 1
        else:
            result.append(char)
    
    return ''.join(result)

In [12]:
test_text2 = "криптография серьезная наука"
encrypted_vigenere = vigenere_cipher(test_text2, password_vigenere)
decrypted_vigenere = vigenere_decipher(encrypted_vigenere, password_vigenere)

In [13]:
print(f"После шифрования:: {encrypted_vigenere}")
print(f"После расшифровки: {decrypted_vigenere}")

После шифрования:: ЦРЪФЮОХШКФФЯГКЬЬЧПЧАЛНТШЦА
После расшифровки: КРИПТОГРАФИЯСЕРЬЕЗНАЯНАУКА
